In [1]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_core.tools import tool
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [2]:
from langchain_core.prompts import ChatPromptTemplate

In [8]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = vector.as_retriever()

In [9]:
retriever.invoke("how to upload a dataset")[0].page_content

'description="A sample dataset in LangSmith.")client.create_examples(    inputs=[        {"postfix": "to LangSmith"},        {"postfix": "to Evaluations in LangSmith"},    ],    outputs=[        {"output": "Welcome to LangSmith"},        {"output": "Welcome to Evaluations in LangSmith"},    ],    dataset_id=dataset.id,)# Define your evaluatordef exact_match(run, example):    return {"score": run.outputs["output"] == example.outputs["output"]}experiment_results = evaluate(    lambda input: "Welcome " + input[\'postfix\'], # Your AI system goes here    data=dataset_name, # The data to predict and grade over    evaluators=[exact_match], # The evaluators to score the results    experiment_prefix="sample-experiment", # The name of the experiment    metadata={      "version": "1.0.0",      "revision_id": "beta"    },)import { Client, Run, Example } from "langsmith";import { evaluate } from "langsmith/evaluation";import { EvaluationResult } from "langsmith/evaluation";const client = new'

In [10]:
from langchain.tools.retriever import create_retriever_tool

In [11]:
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [ ]:
# # Get the prompt to use - can be replaced with any prompt that includes variables "agent_scratchpad" and "input"!
# prompt = hub.pull("hwchase17/openai-tools-agent")
# prompt.pretty_print()

In [12]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)

In [13]:
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [14]:
@tool
def add(first_int: int, second_int: int) -> int:
    "Add two integers."
    return first_int + second_int


@tool
def exponentiate(base: int, exponent: int) -> int:
    "Exponentiate the base to the exponent power."
    return base**exponent



@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

tools = [multiply, add, exponentiate,retriever_tool]
# tools = [search, retriever_tool]

In [15]:
from dotenv import load_dotenv
import os
# Access the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")
load_dotenv()

True

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [17]:
# Construct the tool calling agent
agent = create_tool_calling_agent(llm, tools, prompt)

In [18]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [19]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [20]:
message_history = ChatMessageHistory()
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [21]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm bob"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 41362a5e-af26-4f2e-94c8-a9e1d726f32b not found for run d9456042-7a26-480a-adea-b5e283aa4afd. Treating as a root run.




> Entering new AgentExecutor chain...
Hello Bob! How can I assist you today?

> Finished chain.


{'input': "hi! I'm bob",
 'chat_history': [],
 'output': 'Hello Bob! How can I assist you today?'}

In [22]:
agent_with_chat_history.invoke(
    {"input": "what is 2 times 5 plus 8?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 552febbb-9b6a-48a5-ab00-61eec7947ae2 not found for run d8bf5ebd-3734-4572-bc73-4a139284e97b. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `multiply` with `{'first_int': 2, 'second_int': 5}`


10
Invoking: `add` with `{'first_int': 10, 'second_int': 8}`


182 times 5 is 10, and when you add 8 to that, you get 18.

> Finished chain.


{'input': 'what is 2 times 5 plus 8?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?')],
 'output': '2 times 5 is 10, and when you add 8 to that, you get 18.'}

In [23]:
agent_with_chat_history.invoke(
    {"input": "how can langsmith help with testing?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 4e3f24f6-e470-4083-af4f-093f0c654812 not found for run 745a2bf7-bedd-478b-8594-c8a54db2eb06. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'how can LangSmith help with testing?'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-ope

{'input': 'how can langsmith help with testing?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='what is 2 times 5 plus 8?'),
  AIMessage(content='2 times 5 is 10, and when you add 8 to that, you get 18.')],
 'output': 'LangSmith can help with testing by providing a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application so you can ship quickly and with confidence. LangSmith works independently and does not require LangChain. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n\n3. Set up your environment by exporting necessary variables like `LANGCHAIN_TRACING_V2`, `LANGCHAIN_API_KEY`, and o

In [24]:
agent_with_chat_history.invoke(
    {"input": "how can i setup environment for this?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "abc123"}},
)

Parent run 0bd87b19-cc52-4bde-b1f2-e892cbcbda94 not found for run 3b030969-6ccb-4fe4-8af6-12c0f68bcbaf. Treating as a root run.




> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Setting up environment for LangSmith'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith

Skip to main contentLangSmith API DocsSearchGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith‚ÄãPythonTypeScriptpip install -U langsmithyarn add langchain langsmith2. Create an API key‚ÄãTo create an API key head to the Settings page. Then click Create API Key.3. Set up your environment‚ÄãShellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-ope

{'input': 'how can i setup environment for this?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content='what is 2 times 5 plus 8?'),
  AIMessage(content='2 times 5 is 10, and when you add 8 to that, you get 18.'),
  HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith can help with testing by providing a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application so you can ship quickly and with confidence. LangSmith works independently and does not require LangChain. Here are some steps to get started with LangSmith for testing:\n\n1. Install LangSmith using Python or TypeScript:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langchain langsmith`\n\n2. Create an API key by heading to the Settings page and clicking on Create API Key.\n\n3. Set up your environment by exporting ne